In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys,os
LWNN=os.path.abspath('..')
if(not os.path.isfile('%s/Console.bat'%(LWNN))):
    LWNN=os.path.abspath('.')
p=os.path.abspath('%s/tools'%(LWNN))
sys.path.append(p)
from verifyoutput import *
sys.path.append('C:/Anaconda3/envs/tensorflow/Lib/site-packages')
import tensorflow as tf

%matplotlib inline

In [ ]:
def deconv2d(data, kernel, stride):
    strideX = stride[0]
    strideY = stride[1]
    data = data.astype(np.float64)
    kernel = kernel.astype(np.float64)
    data = data.reshape(1, data.shape[0], data.shape[1], 1)
    kernel = kernel.reshape(kernel.shape[0], kernel.shape[1], 1, 1)
    input = tf.Variable(data)
    filter = tf.Variable(kernel)
    
    input_shape = input.get_shape().as_list()
    filter_shape = filter.get_shape().as_list()
    
    output_shape = [input_shape[0], input_shape[1]*strideX, input_shape[2]*strideY, filter_shape[2]]

    result = None

    op = tf.nn.conv2d_transpose(input, filter, 
                                output_shape=output_shape,
                                strides=[1, strideX, strideY, 1], padding='SAME')
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)

        result = op.eval(session=sess)
    return result

In [ ]:
# configuration
# input data shape
D = (32,32)
# stride 
S = (3,2)
# kernel shape
K = (2,3)

In [ ]:
data = np.random.randint(1,10,size=D)
print(data)

In [ ]:
kernel = np.random.randint(1,10,size=K)
print(kernel)

In [ ]:
result = deconv2d(data, kernel, S)
print(result.reshape(list(result.shape)[1:-1]))

In [ ]:
def deconv2d_ref(data, kernel, oshape, S, PAD):
    Im_in = data.reshape(list(data.shape)+[1])
    kernel = np.rot90(kernel,2)
    print(data)
    print(kernel)
    kernel = kernel.reshape(list(kernel.shape)+[1])
    Im_out = np.zeros(oshape)
    dim_im_in_y,dim_im_in_x,ch_im_in = Im_in.shape
    dim_kernel_y,dim_kernel_x,_ = kernel.shape
    padding_y,padding_x = PAD
    stride_y,stride_x = S
    dim_im_out_y,dim_im_out_x,ch_im_out = oshape
    for i in range(ch_im_out):
        for j in range(dim_im_out_y):
            is_zero = False
            in_row_start = int(j/stride_y)
            of = (j%stride_y)
            kernel_start_y = padding_y - of
            if(kernel_start_y >= 0):
                adj = min(in_row_start, int(kernel_start_y/stride_y))
                kernel_start_y -= adj*stride_y
                in_row_start -= adj
            else:
                adj = -kernel_start_y + dim_kernel_y
                if(adj<=stride_y):
                    is_zero = True
                else:
                    adj = min(dim_im_in_y-1-in_row_start, int(adj/stride_y))
                    kernel_start_y += adj*stride_y
                    in_row_start += adj
            of = dim_kernel_y - 1 - kernel_start_y
            adj = min(dim_im_in_y-1-in_row_start, int(of/stride_y))
            kernel_end_y = kernel_start_y + adj*stride_y
            for k in range(dim_im_out_x):
                conv_out = 0;
                if(is_zero):
                    Im_out[j][k][i] = conv_out
                    continue
                is_zero = False
                in_col_start = int(k/stride_x)
                of = (k%stride_x)
                kernel_start_x = padding_x - of
                if(kernel_start_x >= 0):
                    adj = min(in_col_start, int(kernel_start_x/stride_x))
                    kernel_start_x -= adj*stride_x
                    in_col_start -= adj
                else:
                    adj = -kernel_start_x + dim_kernel_x
                    if(adj<=stride_x):
                        is_zero = True
                    else:
                        adj = min(dim_im_in_x-1-in_col_start, int(adj/stride_x))
                        kernel_start_x += adj*stride_x
                        in_col_start += adj
                of = dim_kernel_x - 1 - kernel_start_x
                adj = min(dim_im_in_x-1-in_col_start, int(of/stride_x))
                kernel_end_x = kernel_start_x + adj*stride_x
                if(is_zero):
                    Im_out[j][k][i] = conv_out
                    is_zero = False
                    continue
                cstr = " out@(%d %d) in(%d %d) kernel@([%d %d],[%d %d]) = "%(
                        j,k, in_row_start,in_col_start,
                        kernel_start_y, kernel_end_y, kernel_start_x, kernel_end_x)
                #print(cstr)
                m = kernel_start_y
                in_row = in_row_start
                while(m <= kernel_end_y):
                    n = kernel_start_x
                    in_col = in_col_start
                    while(n <= kernel_end_x):
                        if ((in_row >= 0) and (in_col >= 0) and
                            (in_row < dim_im_in_y) and (in_col < dim_im_in_x)):
                            for l in range(ch_im_in):
                                conv_out += Im_in[in_row][in_col][l] * kernel[m][n][l]
                                cstr += ' %sx%s + '%(Im_in[in_row][in_col][l], kernel[m][n][l])
                            in_col += 1
                            n+=stride_x
                    in_row += 1
                    m+=stride_y
                Im_out[j][k][i] = conv_out
                cstr += '=%s'%(conv_out)
                #print(cstr)
    return Im_out.reshape(dim_im_out_y, dim_im_out_x)
result2=deconv2d_ref(data, kernel, result.shape[1:], S, (K[0]-1, K[1]-1))
print(result2)
compare(result, result2)